In [ ]:
""" Dialogue Summarization** 경진대회에 오신 여러분 환영합니다!   
본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!
 """

import os
import sys
import datetime

import pandas as pd
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import Dataset , DataLoader

from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig, PreTrainedTokenizerFast
""" from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback """

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
from dotenv import load_dotenv, dotenv_values

import hydra
from omegaconf import DictConfig

# 하이드라와 주피터 노트북은 아규먼트 관련 충돌이 발생하므로 초기화 해줌
sys.argv = ['']
sys.path.append('..')  

from src.data.bart_summarization_dataset import get_datasets
from src import config
from src.models.bart_summarization_module import BartSummarizationModule

def generate_experiment_name(model_name, learning_rate, batch_size, additional_info=""):
    """동적 실험 이름 생성"""
    timestamp = datetime.datetime.now().strftime("%m%d_%H%M")
    
    base_name = f"{model_name.replace('/', '_')}_lr{learning_rate}_bs{batch_size}"
    
    if additional_info:
        base_name += f"_{additional_info}"
    
    return f"{base_name}_{timestamp}"    


# 데이터 준비 함수
def prepare_data(cfg, tokenizer):
    
   # 데이터셋 생성
    train_dataset, val_dataset, test_dataset = get_datasets(cfg, tokenizer)

    """ print(len(train_dataset))
    print(len(val_dataset))
    print(len(test_dataset))
    sample_0 = train_dataset[0]
    print(sample_0) """


    # DataLoader 정의
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.data.batch_size,
        shuffle=True,
        num_workers=cfg.data.num_workers,
        pin_memory=True,
        drop_last=False
    )

    val_loader = DataLoader(
        val_dataset,  # 별도의 검증 데이터셋
        batch_size=cfg.data.batch_size,
        shuffle=False,  # 검증 시에는 셔플하지 않음
        num_workers=cfg.data.num_workers,
        pin_memory=True,
        drop_last=False
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.data.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False
    )
    
    return train_loader, val_loader, test_loader

# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(cfg):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {cfg.model.model_name}', '-'*10,)
    model_name = cfg.model.model_name
    
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(model_name,config=bart_config)
    #tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
    #generate_model = BartForConditionalGeneration.from_pretrained(model_name)

    special_tokens_dict={'additional_special_tokens':[str(token) for token in cfg.tokenizer.special_tokens]}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer


def test(trainer, model, test_loader):

    # 테스트
    trainer.test(model, test_loader)

    print("테스트 갯수=",len(model.test_outputs))
    
    if len(model.test_outputs) > 0:
        # 모든 예측값과 실제값 합치기
        all_preds = model.test_outputs
        pred_df = test_loader.dataset.df

        output = pd.DataFrame(
            {
                "fname": pred_df['fname'],
                "summary" : all_preds,
            }
        )

        output.to_csv("output.csv", index=False)
        
    else:
        print("테스트 결과를 가져올 수 없습니다.")


@hydra.main(config_path=str(config.CONFIGS_DIR), config_name="config", version_base=None)
def main(cfg):

     # cfg 에서 최상위 model 키값을 뺀다.
    cfg = cfg.model

    # WandB Logger 초기화
    wandb_logger = WandbLogger(
        project= str(dotenv_values().get('WANDB_PROJECT')),                                                                                        # 프로젝트 이름
        name=generate_experiment_name(cfg.model.model_name, cfg.optimizer.lr, cfg.data.batch_size),                   # 실험 이름 (선택사항)
        job_type="train",                                                                                                   # 작업 타입 (선택사항)
        save_dir=str(config.LOG_ROOT_DIR),
        log_model=True

    )

    pl.seed_everything(cfg.custom.seed_num)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model , tokenizer = load_tokenizer_and_model_for_train(cfg)
    print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 데이터 로더 준비
    train_loader, val_loader, test_loader = prepare_data(cfg, tokenizer)


    # Lightning Module 생성
    lightning_module = BartSummarizationModule(cfg, generate_model, tokenizer)

    # 콜백을 직접 생성
    early_stopping = hydra.utils.instantiate(cfg.callbacks.early_stopping)
    lr_monitor = hydra.utils.instantiate(cfg.callbacks.lr_monitor)
    model_checkpoint = hydra.utils.instantiate(cfg.callbacks.model_checkpoint)

    callbacks = [early_stopping, lr_monitor, model_checkpoint]

    trainer = Trainer(
        default_root_dir=str(config.CHECKPOINT_DIR), 
        max_epochs=cfg.trainer.max_epochs, 
        accelerator=cfg.trainer.accelerator, 
        callbacks=callbacks, 
        logger=wandb_logger, 
        num_sanity_val_steps=cfg.trainer.num_sanity_val_steps)
    

    # 훈련
    if cfg.custom.do_checkpoint == True and os.path.exists(cfg.custom.ckpt_path):
        trainer.fit(lightning_module, train_loader, val_loader, ckpt_path=cfg.custom.ckpt_path)
    else:
        trainer.fit(lightning_module, train_loader, val_loader)
    
    # 테스트
    if(cfg.custom.do_test == True): 
        test(trainer, lightning_module, test_loader)


    wandb.finish()

if __name__ == "__main__":

    main()

In [13]:
import random
from gensim.models import FastText
from kiwipiepy import Kiwi
import re

# --- 1. 모델과 형태소 분석기 로드 ---
try:
    # FastText 모델 로드
    ft_model = FastText.load_fasttext_format('/data/ephemeral/home/work/python/upstageailab-nlp-summarization-nlp_3/cc.ko.300.bin')
    word_vectors = ft_model.wv # 벡터 부분만 추출
    
    # Kiwi 형태소 분석기 로드
    kiwi = Kiwi()
    
    print("성공: FastText 모델과 Kiwi 형태소 분석기를 모두 로드했습니다.")

except Exception as e:
    print(f"모델 또는 분석기 로딩 중 에러 발생: {e}")
    word_vectors = None
    kiwi = None


def get_synonyms(
    word: str, 
    wv_model, 
    topn: int = 10,
    score_threshold: float = 0.65
) -> list:
    """
    단어와 모델을 받아, 여러 품질 관리 가이드라인을 통과한
    '고품질의 한글 유의어' 리스트만 반환합니다.
    """
    
    if wv_model is None or word not in wv_model:
        return []
    
    similar_word_pairs = wv_model.most_similar(word, topn=topn)
    
    qualified_synonyms = []
    
    for synonym, score in similar_word_pairs:
        # 가이드라인 1: 유사도 점수가 기준보다 낮으면 탈락
        if score < score_threshold:
            continue
            
        # 가이드라인 2: 원본과 글자 수가 너무 차이 나면 탈락 (간단한 버전)
        if len(word) == 1 and len(synonym) > 2:
            continue

        # ★★★ 새로 추가된 가이드라인 3: 한글로만 이루어져 있는지 확인 ★★★
        # 정규표현식을 사용하여, 단어가 처음부터 끝까지 한글로만 구성되었는지 검사합니다.
        if not re.match("^[가-힣]+$", synonym):
            continue
            
        qualified_synonyms.append(synonym)
        
    return qualified_synonyms

# ★★★ 핵심: Kiwi를 사용하는 최종 버전의 증강 함수 ★★★
def synonym_replacement_eda_final(sentence: str, wv_model, kiwi_analyzer, augmentation_ratio: float = 0.2) -> str:
    if wv_model is None or kiwi_analyzer is None:
        return sentence

    morphemes = kiwi_analyzer.analyze(sentence)[0][0]
    
    candidate_indices = [i for i, token in enumerate(morphemes) if token.tag in ['NNG', 'NNP', 'VV', 'VA']]
    
    if not candidate_indices:
        return sentence

    num_to_replace = int(len(candidate_indices) * augmentation_ratio)
    if num_to_replace == 0:
        num_to_replace = 1 # 최소 1개는 바꾸도록 설정

    indices_to_replace = random.sample(candidate_indices, k=min(num_to_replace, len(candidate_indices)))
    
    augmented_morphemes = list(morphemes)

    for i in indices_to_replace:
        original_morpheme = morphemes[i]
        # 기본형(어간)으로 유의어 탐색
        synonyms = get_synonyms(original_morpheme.form, wv_model)
        
        if synonyms:
            synonym = random.choice(synonyms)
            augmented_morphemes[i] = (synonym, original_morpheme.tag, original_morpheme.start, original_morpheme.len)
            
    # --- ★★★ 여기가 수정된 부분입니다! ★★★ ---
    # kiwi.join이 요구하는 (단어, 품사) 튜플 리스트를 새로 만들어줍니다.
    morphs_for_join = [(m[0], m[1]) for m in augmented_morphemes]

    # 새로 만든, 형식이 맞는 리스트를 전달합니다.
    return kiwi_analyzer.join(morphs_for_join)


# --- 3. 최종 함수 호출 ---
if word_vectors and kiwi:
    original_text = "의사가 환자에게 입원할것을 권합니다"
    # 최종 버전의 함수를, word_vectors와 kiwi 객체와 함께 호출
    augmented_text = synonym_replacement_eda_final(original_text, wv_model=word_vectors, kiwi_analyzer=kiwi)

    print("-" * 20)
    print(f"원본 문장: {original_text}")
    print(f"증강 문장: {augmented_text}")
    # 예상 출력 -> 증강 문장: 의원이 환자에게 가볼 것을 추천합니다 (예시)


/tmp/ipykernel_373315/1929248924.py:9: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  ft_model = FastText.load_fasttext_format('/data/ephemeral/home/work/python/upstageailab-nlp-summarization-nlp_3/cc.ko.300.bin')


성공: FastText 모델과 Kiwi 형태소 분석기를 모두 로드했습니다.
--------------------
원본 문장: 의사가 환자에게 입원할것을 권합니다
증강 문장: 의사가 환자에게 입원할 것을 권합니다
